## 1. Download and export the raw patent data

In [1]:
df_patent = pd.read_csv('../Network data/pub_patent_edges.csv', dtype = 'string')

NameError: name 'pd' is not defined

In [33]:
# make list of patentIDs (got to include the double quotes); delete duplicates
data = json.dumps(list(set(df_patent['patentID'])))

In [5]:
len(list(set(df_patent['patentID'])))

1315

In [30]:
## for testing the scraper on one patent
# I am not sure about the legal status of these patents (are they still mantained?)

data_dict = []
start_time = time.time()

url = 'https://api.lens.org/patent/search'
data_patent = '''{
              "query": {
                  "terms":  {
                      "lens_id": ["013-408-491-874-758"]
                              }
                          },
              "include": ["doc_key","biblio.parties", "biblio.invention_title", "biblio.references_cited", "biblio.cited_by", "jurisdiction", "date_published", "publication_type"]
            
            }''' 
headers = {'Authorization': 'IjpxxGZZQcGCQxBzAx5VDqPqHXhQjAqSkdnAYHQCRAkfkAlVDjCI', 'Content-Type': 'application/json'}
response = requests.post(url, data=data_patent, headers=headers)
if response.status_code != requests.codes.ok:
    print(response.status_code)

else:
    print(response.json)
data_dict.append(response.json())

print("--- %s seconds ---" % (time.time() - start_time))

<bound method Response.json of <Response [200]>>
--- 0.9220991134643555 seconds ---


In [26]:
data_dict

[{'total': 1,
  'data': [{'lens_id': '104-626-853-296-452',
    'jurisdiction': 'WO',
    'date_published': '2021-12-16',
    'doc_key': 'WO_2021250546_A1_20211216',
    'biblio': {'invention_title': [{'text': 'MICRORNAS AS PREDICTORS OF RESPONSE TO ANTI-IGE THERAPIES IN CHRONIC SPONTANEOUS URTICARIA',
       'lang': 'en'},
      {'text': "MICRO-ARN EN TANT QUE PRÉDICTEURS DE RÉPONSE À DES THÉRAPIES ANTI-IGE DANS L'URTICAIRE CHRONIQUE SPONTANÉE",
       'lang': 'fr'}],
     'parties': {'applicants': [{'residence': 'US',
        'extracted_name': {'value': 'UNIV WASHINGTON'}}],
      'inventors': [{'residence': 'US',
        'sequence': 1,
        'extracted_name': {'value': 'HENDERSON WILLIAM R'}},
       {'residence': 'US',
        'sequence': 2,
        'extracted_name': {'value': 'MACDONALD JAMES W'}},
       {'residence': 'US',
        'sequence': 3,
        'extracted_name': {'value': 'BAMMLER THEODOR K'}},
       {'residence': 'US',
        'sequence': 4,
        'extracted_name'

In [34]:
data_dict = []
url = 'https://api.lens.org/patent/search'

# include fields
include = '''["doc_key","biblio.parties", "biblio.invention_title", "biblio.references_cited", "biblio.cited_by", "jurisdiction", "date_published", "publication_type"]'''


# request body with scroll time of 1 minute
request_body = '''{
  "query": {
      "terms":  {
          "lens_id": %s
      }
  },
  "include": %s
  , "size": 100
  , "scroll": "1m"
}''' % (data, include)
headers = {'Authorization': 'IjpxxGZZQcGCQxBzAx5VDqPqHXhQjAqSkdnAYHQCRAkfkAlVDjCI', 'Content-Type': 'application/json'}


# Recursive function to scroll through paginated results
def scroll(scroll_id):
  # Change the request_body to prepare for next scroll api call
  # Make sure to append the include fields to make faster response
    if scroll_id is not None:        
        global request_body
        request_body = '''{"scroll_id": "%s", "include": %s}''' % (scroll_id, include)
        
    # make api request
    response = requests.post(url, data=request_body, headers=headers) 
    
    # If rate-limited, wait for n seconds and proceed the same scroll id
    # Since scroll time is 1 minutes, it will give sufficient time to wait and proceed
    if response.status_code == requests.codes.too_many_requests:
        time.sleep(8)
        scroll(scroll_id)
    # If the response is not ok here, better to stop here and debug it
    
    elif response.status_code != requests.codes.ok:
        print(response.json())
        print('Äahh')
    # If the response is ok, do something with the response, take the new scroll id and iterate
    
    # leave the function if it returns empty (because then everything is downloaded already)
    json = response.json()
    if json['data'] == []:
        return
    
    else:
        json = response.json()
        scroll_id = json['scroll_id'] # Extract the new scroll id from response
        #print(json['data']) #DO something with your data
        data_dict.append(response.json())
        
        scroll(scroll_id)
        
# start recursive scrolling
scroll(scroll_id=None)

In [35]:
# check out the entries
data_dict[1]['data'][1]['lens_id']

'001-535-333-300-333'

In [37]:
#compare list of stored ids with pervious list 
d = []
for n in data_dict: 
    #d = d+len(n['data'])
    for i in n['data']:
        d.append(i['lens_id'])
        
for i in list(set(df_patent['patentID'])):
    if i not in d: 
        print(i)

177-970-513-795-707
168-177-908-076-053
152-391-069-052-806
023-825-995-431-07X
081-101-592-084-931
010-439-583-636-995
020-946-286-258-253
046-826-735-829-232
002-541-007-620-216
013-896-450-318-443
059-578-735-298-090
105-178-793-365-769
010-250-529-230-290
101-329-421-538-77X
190-797-046-811-919
190-240-428-371-869
069-605-151-180-041
107-119-408-136-453
019-169-055-130-421
161-876-210-019-928
013-408-491-874-758
045-405-085-214-807


In [ ]:
# for some reason lens uses other IDs for these... could be a good question to ask

In [11]:
len(d)

1315

In [39]:
# save it
with open(r'..\Raw data\full-data_patent-lens.json', 'w') as f:
    json.dump(data_dict, f)

In [ ]:
# 3. Patents

### 3.1. Download and export the raw patent data

df_patent = pd.read_csv('../Network data/pub_patent_edges.csv', dtype = 'string')

# make list of patentIDs (got to include the double quotes); delete duplicates
data = json.dumps(list(set(df_patent['patentID'])))

len(list(set(df_patent['patentID'])))

## for testing the scraper on one patent
# I am not sure about the legal status of these patents (are they still mantained?)

data_dict = []
start_time = time.time()

url = 'https://api.lens.org/patent/search'
data_patent = '''{
              "query": {
                  "terms":  {
                      "lens_id": ["013-408-491-874-758"]
                              }
                          },
              "include": ["doc_key","biblio.parties", "biblio.invention_title", "biblio.references_cited", "biblio.cited_by", "jurisdiction", "date_published", "publication_type"]
            
            }''' 
headers = {'Authorization': 'IjpxxGZZQcGCQxBzAx5VDqPqHXhQjAqSkdnAYHQCRAkfkAlVDjCI', 'Content-Type': 'application/json'}
response = requests.post(url, data=data_patent, headers=headers)
if response.status_code != requests.codes.ok:
    print(response.status_code)

else:
    print(response.json)
data_dict.append(response.json())

print("--- %s seconds ---" % (time.time() - start_time))

data_dict

data_dict = []
url = 'https://api.lens.org/patent/search'

# include fields
include = '''["doc_key","biblio.parties", "biblio.invention_title", "biblio.references_cited", "biblio.cited_by", "jurisdiction", "date_published", "publication_type"]'''


# request body with scroll time of 1 minute
request_body = '''{
  "query": {
      "terms":  {
          "lens_id": %s
      }
  },
  "include": %s
  , "size": 100
  , "scroll": "1m"
}''' % (data, include)
headers = {'Authorization': 'IjpxxGZZQcGCQxBzAx5VDqPqHXhQjAqSkdnAYHQCRAkfkAlVDjCI', 'Content-Type': 'application/json'}


# Recursive function to scroll through paginated results
def scroll(scroll_id):
  # Change the request_body to prepare for next scroll api call
  # Make sure to append the include fields to make faster response
    if scroll_id is not None:        
        global request_body
        request_body = '''{"scroll_id": "%s", "include": %s}''' % (scroll_id, include)
        
    # make api request
    response = requests.post(url, data=request_body, headers=headers) 
    
    # If rate-limited, wait for n seconds and proceed the same scroll id
    # Since scroll time is 1 minutes, it will give sufficient time to wait and proceed
    if response.status_code == requests.codes.too_many_requests:
        time.sleep(8)
        scroll(scroll_id)
    # If the response is not ok here, better to stop here and debug it
    
    elif response.status_code != requests.codes.ok:
        print(response.json())
        print('Äahh')
    # If the response is ok, do something with the response, take the new scroll id and iterate
    
    # leave the function if it returns empty (because then everything is downloaded already)
    json = response.json()
    if json['data'] == []:
        return
    
    else:
        json = response.json()
        scroll_id = json['scroll_id'] # Extract the new scroll id from response
        #print(json['data']) #DO something with your data
        data_dict.append(response.json())
        
        scroll(scroll_id)
        
# start recursive scrolling
scroll(scroll_id=None)

# check out the entries
data_dict[1]['data'][1]['lens_id']

#compare list of stored ids with pervious list 
d = []
for n in data_dict: 
    #d = d+len(n['data'])
    for i in n['data']:
        d.append(i['lens_id'])
        
for i in list(set(df_patent['patentID'])):
    if i not in d: 
        print(i)

# for some reason lens uses other IDs for these... could be a good question to ask

len(d)

# save it
with open(r'..\Raw data\full-data_patent-lens.json', 'w') as f:
    json.dump(data_dict, f)